In [2]:
import pandas as pd

In [3]:
county_data = pd.read_csv('county_demographics_coding_task.csv')

In [4]:
# share of races, ethnicities, and gender per county per year

county_data['share_white'] = county_data['white']/county_data['population']
county_data['share_black'] = county_data['black']/county_data['population']
county_data['share_asian'] = county_data['asian']/county_data['population']
county_data['share_natam'] = county_data['natam']/county_data['population']
county_data['share_hipi'] = county_data['hipi']/county_data['population']
county_data['share_other_race'] = county_data['other_race']/county_data['population']
county_data['share_two_or_more'] =county_data['black']/county_data['population']
county_data['share_hispanic'] = county_data['hispanic']/county_data['population']
county_data['share_white_not_hispanic'] = county_data['white_not_hispanic']/county_data['population']
county_data['share_male'] = county_data['male']/county_data['population']
county_data['share_female'] = county_data['female']/county_data['population']


county_data.to_csv('/Users/divya/Documents/ra_data_task/data/cleaned_county_data.csv')


In [20]:
# In the average county, what share of the county’s population lives in the one district that a plurality of the county’s population is in?

# group by county and find the average share of population in a plurality district across the years of analysis
average_county_share = county_data

#isolate data to counties with multiple districts
multiple_district_counties = average_county_share[['cd_share_of_county','single_district_county','county']]
multiple_district_counties = multiple_district_counties[multiple_district_counties['single_district_county'] !=1]
print(multiple_district_counties)





multiple_district_counties = multiple_district_counties.groupby('county')['cd_share_of_county'].mean().reset_index(name = 'avg_share_in_plurality')
print(multiple_district_counties)
# find the average of average county population shares
multiple_district_counties = multiple_district_counties['avg_share_in_plurality'].mean()

multiple_district_counties


       cd_share_of_county  single_district_county  county
83               0.794674                       0    1007
84               0.915243                       0    1009
85               0.915243                       0    1009
86               0.915243                       0    1009
87               0.915243                       0    1009
...                   ...                     ...     ...
64743            0.590547                       0   55141
64744            0.590559                       0   55141
64745            0.590559                       0   55141
64746            0.590559                       0   55141
64747            0.590559                       0   55141

[9183 rows x 3 columns]
     county  avg_share_in_plurality
0      1007                0.794674
1      1009                0.915210
2      1019                0.932041
3      1025                0.615927
4      1037                0.738613
..      ...                     ...
748   55117                

np.float64(0.7157762412195273)

In [54]:
# share of people living in a county's main district 

# average share of ppl living in each county's main district
avg_share_per_county = county_data.groupby('county')['cd_share_of_county'].mean()

# median population per county
median_population_per_county = pd.DataFrame()
median_population_per_county['median_population'] = county_data.groupby('county')['population'].median()
total_population = median_population_per_county['median_population'].sum()

# amount of ppl living in county's main district across all years and counties

# finds AMOUNT of ppl living in main district per county
ppl_main_district_per_county = avg_share_per_county*median_population_per_county['median_population']


# sums all ppl living in main district 
total_ppl_main_district = ppl_main_district_per_county.sum()

# calculates total share of ppl living in a main district
total_share = total_ppl_main_district/total_population

total_share






np.float64(0.7121545236315586)

In [21]:
# table comparing race, ethnicity, gender, and whether the county is in a single district or not
county_demogrph = pd.DataFrame()
county_demogrph['race/ethnicity/gender'] = ['share_white', 'share_black', 'share_natam', 'share_asian','share_hipi', 'share_other_race', 'share_two_or_more', 'share_hispanic', 'share_white_not_hispanic', 'share_male', 'share_female']


# find median race/gender/ethnicity share per county in single districts counties.
single_district_counties = county_data[ county_data['single_district_county']==1]

race_medians_per_sdc = single_district_counties.groupby('county')[['share_white', 'share_black', 'share_natam', 'share_asian', 'share_hipi', 'share_other_race', 'share_two_or_more', 'share_hispanic', 'share_white_not_hispanic', 'share_male', 'share_female']].median()

# finds median share across all single county districts (scd)
medians_all_sdc = race_medians_per_sdc.median()

# Now, we map the median values from medians_series to district_demogrph
county_demogrph['single_district_counties_share'] = county_demogrph['race/ethnicity/gender'].map(medians_all_sdc)



# find median race/gender/ethnicity share per county in multiple districts counties.
    

multi_district_counties = county_data[ county_data['single_district_county']==0]

race_medians_per_mdc = multi_district_counties .groupby('county')[['share_white', 'share_black', 'share_natam', 'share_asian', 'share_hipi', 'share_other_race', 'share_two_or_more', 'share_hispanic', 'share_white_not_hispanic', 'share_male', 'share_female']].median()

# finds median share across all single county districts (scd)
medians_all_mdc = race_medians_per_mdc.median()


# map the median values in medians_all_mdc to district_demogrph
county_demogrph['multiple_district_counties_share'] = county_demogrph['race/ethnicity/gender'].map(medians_all_mdc)

# find median race/gender/ethnicity population for each county 
median_demogrphs_per_county = (county_data.groupby('county')[['white', 'black', 'natam', 'asian', 'hipi', 'other_race', 'two_or_more', 'hispanic', 'white_not_hispanic', 'male', 'female']]).median()

# sum medians across each county
total_demogrphs_all = median_demogrphs_per_county[['white', 'black', 'natam', 'asian', 'hipi', 'other_race', 'two_or_more', 'hispanic', 'white_not_hispanic', 'male', 'female']].sum()


median_population_per_county = county_data.groupby('county')['population'].median().reset_index(name = 'median')


total_population = median_population_per_county['median'].sum()


total_shares = (total_demogrphs_all.div(total_population)).to_frame().reset_index()

total_shares = total_shares.rename(columns={'index': 'race/ethnicity/gender'})
total_shares = total_shares.rename(columns={0: 'total_share'})



county_demogrph['share_across_all_counties'] = total_shares['total_share']


county_demogrph.to_csv('/Users/divya/Documents/ra_data_task/data/cleaned_county_data.csv', index=False)

county_demogrph





,race/ethnicity/gender,single_district_counties_share,multiple_district_counties_share,share_across_all_counties
0,share_white,0.900528,0.823620,0.707176
1,share_black,0.016745,0.060690,0.125476
2,share_natam,0.003906,0.003490,0.009177
3,share_asian,0.004134,0.009230,0.046045
4,share_hipi,0.000270,0.000343,0.001281
5,share_other_race,0.010872,0.017794,0.063663
6,share_two_or_more,0.016745,0.060690,0.028753
7,share_hispanic,0.029183,0.047285,0.164281
8,share_white_not_hispanic,0.873904,0.775588,0.637047
9,share_male,0.495379,0.490971,0.491233
